<a href="https://colab.research.google.com/github/hwwbg/DECIS-DataScientist/blob/main/2-advanced-topics/geospatial-analysis/geospatial-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Geospatial Analysis in Python

## Loading and exploring spatial data

### Setup

The main geospatial packages that we'll load are __Shapely__ and __GeoPandas__.

* "__[Shapely](https://pypi.org/project/shapely/)__ is a BSD-licensed Python package for manipulation and analysis of planar geometric objects"
* "__[GeoPandas](https://geopandas.org/en/stable/)__ extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely."

In short, __GeoPandas__ allows processing tabular data (like Pandas), where each row is associated with a geometry---and where the geometry is defined using __Shapely__.

Shapely defines multiple geometry types:

![](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/shapely_geom_types.png?raw=1)

__Source:__ [Shapely](https://isbe.bwk.tue.nl/education/Python/04_02_Shapely.html#fundamental-geometric-objects)

In [1]:
# If you run this notebook in your computer, you might need to install these packages:
# !pip install geopandas   # <-- pandas like dataframes with shapely geometries
# !pip install folium      # <-- interactive/leaflet maps
# !pip install shapely     # <-- manipulate/analysis of geometric objects
# !pip install numpy       # <-- scientific computing

# In Colab, need to install these
!pip install gadm        # <-- download data from GADM
!pip install osmnx       # <-- download/clean data from OpenStreetMaps (OSM)
!pip install mapclassify # <-- facilitates making choropleth maps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.8/994.8 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 70.7 MB/s eta 0:00:00
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681924 sha256=387dee13768f72f7624eff2e6696ff501a7b79140ef7566888bda18ee6f709e1
  Stored in directory: /root/.cache/pip/wheels/cd/29/8b/617685ed7942656b36efb06ff9247dbe832e3f4f7724fffc09
Successfully built pycountry
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstal

In [2]:
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from gadm import GADMDownloader
import osmnx as ox
from geopy.distance import geodesic

In [ ]:
# If it doesn't already exist, create a directory to put data
if not os.path.exists("data"):
    os.mkdir("data")

### Load and explore polygon

In [ ]:
# Download administrative region polygons from GADM

# Skip if data already exist
if not os.path.exists(os.path.join("data", "gadm_nga_2.geojson")):

    # Load downloader object, using GADM version 4.0
    downloader = GADMDownloader(version="4.0")

    # Download data and save to data directory
    nga_0 = downloader.get_shape_data_by_country_name(country_name="Nigeria", ad_level=0)
    nga_0.to_file(os.path.join("data", "gadm_nga_0.geojson"), driver="GeoJSON")

    nga_1 = downloader.get_shape_data_by_country_name(country_name="Nigeria", ad_level=1)
    nga_1.to_file(os.path.join("data", "gadm_nga_1.geojson"), driver="GeoJSON")

    nga_2 = downloader.get_shape_data_by_country_name(country_name="Nigeria", ad_level=2)
    nga_2.to_file(os.path.join("data", "gadm_nga_2.geojson"), driver="GeoJSON")

In [ ]:
nga2_gdf = gpd.read_file(os.path.join("data", "gadm_nga_2.geojson"))

In [ ]:
nga2_gdf.head()

In [ ]:
nga2_gdf.shape

In [ ]:
nga2_gdf.plot()

In [ ]:
nga2_gdf.crs

In [ ]:
nga2_gdf.explore()

#### Area of each polygon

In [ ]:
# Are is not a variable :(
"area" in nga2_gdf.columns.tolist()

In [ ]:
# But we can still figure out the area :)
nga2_gdf.area

In [ ]:
# Lets add the area as a variable
nga2_gdf['area'] = nga2_gdf.area

#### Operations common to pandas dataframes also work on geodataframes

In [ ]:
lagos_gdf = nga2_gdf[nga2_gdf['NAME_1'] == 'Lagos']

In [ ]:
lagos_gdf.plot()

### Exercise 1: Load and Explore Polyline

In [ ]:
## Extracting road network data for Lagos from OpenStreetMaps data

# Skip if data already exists
if not os.path.exists(os.path.join("data", "osm_lagos_roads.geojson")):

    #### One geometry for Lagos
    nga1_gdf = gpd.read_file(os.path.join("data", "gadm_nga_1.geojson"))
    lagos_1_gdf = nga1_gdf[nga1_gdf['NAME_1'] == 'Lagos']

    #### Grab larger roads from OSM; ignoring unclassified and residential
    roads_gdf = ox.features.features_from_polygon(lagos_1_gdf.geometry.iloc[0],
                                                        tags = {'highway':['motorway',
                                                                           'trunk',
                                                                           'primary',
                                                                           'secondary',
                                                                           'tertiary']})

    #### Cleanup
    roads_gdf = roads_gdf.reset_index()

    #### Export
    roads_gdf.to_file(os.path.join("data", "osm_lagos_roads.geojson"), driver="GeoJSON")

#### 1a: Load data

Load the roads data `data/osm_lagos_roads.geojson`, and name the object `roads_gdf`

#### 1b. Look at the first few observations of the data

#### 1c: What is the coordinate reference system of the data?

#### 1d: Make a static map of the road network

#### 1e: Make an interactive map of the road network that just includes motorways and trunk roads

#### 1f: Add a variable indicating the length of each road

### Load and Explore Point Data

In [ ]:
## Extracting school data for Lagos from OpenStreetMaps data

##: NOTE: Warnings may appear, but that's OK!

# Skip if data already exists
if not os.path.exists(os.path.join("data", "osm_lagos_schools.csv")):

    #### One geometry for Lagos
    nga1_gdf = gpd.read_file(os.path.join("data", "gadm_nga_1.geojson"))
    lagos_1_gdf = nga1_gdf[nga1_gdf['NAME_1'] == 'Lagos']

    #### Grab larger roads from OSM; ignoring unclassified and residential
    schools_gdf = ox.features.features_from_polygon(lagos_1_gdf.geometry.iloc[0],
                                                        tags = {'amenity':['school']})

    #### Cleanup
    schools_gdf = schools_gdf.reset_index()

    #### Original is polygon; convert to points using polygon centroid
    schools_gdf['geometry'] = schools_gdf['geometry'].centroid

    #### Add latitude and longitude variables based on centroid
    # (Remove existing Lat/Lon variables, some of which are NA)
    schools_gdf = schools_gdf.drop(['Latitude', 'Longitude'], axis = 1)

    schools_gdf['latitude'] = schools_gdf['geometry'].y
    schools_gdf['longitude'] = schools_gdf['geometry'].x

    #### Drop geometry; convert from geodataframe to dataframe
    schools_df = schools_gdf.drop('geometry', axis=1)

    #### Export
    schools_df.to_csv(os.path.join("data", "osm_lagos_schools.csv"), index = False)

#### Load school data

In [ ]:
schools_df = pd.read_csv(os.path.join("data", "osm_lagos_schools.csv"))

In [ ]:
schools_df.head()

#### Convert from dataframe to geodataframe

In [ ]:
# Combine 'latitude' and 'longitude' columns to create a GeoSeries of Point geometries
geometry = [Point(lon, lat) for lon, lat in zip(schools_df['longitude'], schools_df['latitude'])]
geo_series = gpd.GeoSeries(geometry)

# Convert the Pandas DataFrame to a GeoPandas GeoDataFrame
schools_gdf = gpd.GeoDataFrame(schools_df, geometry=geo_series, crs = 4326)

#### Explore data

In [ ]:
schools_gdf.head()

In [ ]:
schools_gdf.crs

In [ ]:
schools_gdf.explore()

#### Exercise 2: Assign the wrong CRS and see what happens

Above we used the following code to convert the dataframe to a GeoDataFrame, by indicating (1) the data, (2) the geometry, and (3) the CRS:

`schools_gdf = gpd.GeoDataFrame(schools_df, geometry=geo_series, crs = 4326)`

Now, we'll explore what happens if we assign the wrong CRS.

#### 2a. Create an object with the wrong CRS

Adapt the code just above to create a new object called `schools_bad_gdf`; use the CRS `3857`

#### 2b. Make a static map of `schools_bad_gdf`. From this, can we tell that there may be an issue with the CRS?

#### 2b. Make an interactive map of `schools_bad_gdf`. From this, can we tell that there may be an issue with the CRS?

### Maps

In [ ]:
# This is boring.
lagos_gdf.plot()

In [ ]:
# Pretty :)
lagos_gdf.plot(column='area',
               cmap='Spectral',
               legend=True,
               figsize = (20,5))

In [ ]:
lagos_gdf.explore(
     column="area", # make choropleth based on "pings" column
     tooltip="area", # show pings value when hover over
     cmap="Spectral", # use "Spectral" matplotlib colormap
     style_kwds=dict(color="black") # use black outline
    )

#### Exercise 3: Make a different version of the interactive map

1. Use a different color palette (for options, see [here](https://matplotlib.org/stable/users/explain/colors/colormaps.html)).
2. When hovering over ADM2 regions, show the name of the region (not the area)

### Spatial Operations: Applied on Single Dataset

#### Transform CRS

We want to compute the length of roads in Lagos using the roads dataset. The roads dataset is currently in a geographic CRS (WGS84), where the units are in decimal degrees. We'll tranform the CRS to a __projected__ CRS that is suitable for Nigeria ([EPSG:32632](https://epsg.io/32632)), and where the units will be in meters.

In [ ]:
# Load data
roads_gdf = gpd.read_file(os.path.join("data", "osm_lagos_roads.geojson"))

In [ ]:
# Length is in decimal degrees. Unhelpful!
roads_gdf.length.head()

In [ ]:
# Use to_crs to change the CRS
roads_newcrs_gdf = roads_gdf.to_crs(32632)

In [ ]:
# Now the length is in meters!
roads_newcrs_gdf.length.head()

Let's look at the correlation between the length in decimal degrees and the length in meters. The correlation super close to 1, but is not exactly 1---demonstrating that the CRS matters when computing lengths, distances, areas, etc. Remember, as latitude changes, the kilometer distance between 1 degree longitude changes. Lagos is all at a very similar latitude, so we'd expect the correlation between length caculated from a geographic and projected CRS to be similar.

In [ ]:
import numpy
numpy.corrcoef(roads_newcrs_gdf.length.to_list(),
               roads_gdf.length.to_list())[0,1]

#### Exercise 4: Transform the CRS of the Nigeria ADM2 object

#### 4a: Transform the CRS of the Nigeria ADM2 object `nga2_gdf` to EPSG:32632

#### 4b: Determine the area in decimal degrees and meters squared

#### 4c: Compute the correlation between the area in decimal degrees and meters squared

#### Buffer

We have the points of schools. Now we create a 1km buffer around schools

In [ ]:
schools_1km_gdf = schools_gdf.copy()
schools_1km_gdf = schools_1km_gdf.to_crs(32632)

schools_1km_gdf['geometry'] = schools_1km_gdf.geometry.buffer(1000)

In [ ]:
schools_1km_gdf.explore()

#### Dissolve by an Attribute

Below we have the second administrative regions of Nigeria. Using this dataset, let's create a new object at the first administrative region level.

In [ ]:
nga1_simple_gdf = nga2_gdf.dissolve('NAME_1')

In [ ]:
nga2_gdf.plot()

In [ ]:
nga1_simple_gdf.plot()

#### Convex Hull

__Simple definition:__ Get the outer-most coordinates of a shape and connect-the-dots.

__Fomal definition:__
A convex hull of a shape the smallest "convex set" that contains it. (A [convex set](https://en.wikipedia.org/wiki/Convex_set) is where a straight line can be drawn anywhere in the space and the space fully contains the line).

Convex           |  Not Convex
:-------------------------:|:-------------------------:
![convex](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/Convex_polygon_illustration1.svg.png?raw=1)  |  ![notconvex](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/220px-Convex_polygon_illustration2.svg.png?raw=1)

__Source:__ [Wikipedia](https://en.wikipedia.org/wiki/Convex_set)

In the below example, we create a conex hull around schools; creating a polygon that includes all schools.

__Incorrect attempt:__ We use `convex_hull`, which applies a convex hull to each row. Because schools are points, the resulting geometries are the same (they're still points!)

In [ ]:
schools_gdf['c_hull'] = schools_gdf.convex_hull

schools_gdf['c_hull'].plot()

__Correct attempt:__ We first dissolve the schools into one geometry, then compute the convex hull.

In [ ]:
schools_gdf['id_temp'] = 1
schools_diss_gdf = schools_gdf.dissolve('id_temp')

schools_diss_gdf['c_hull'] = schools_diss_gdf.convex_hull

schools_diss_gdf['c_hull'].plot()

In [ ]:
# Note that the geometry type is a "multipoint"
schools_diss_gdf.head()

#### Determine Centroid

Sometimes we want to represent a polygon or polyline as a single point. For this, we can compute the __centroid__ (ie, geographic center) of a polygon/polyline.

![centroid](https://github.com/worldbank/dec-python-course/blob/main/1-foundations/5-geospatial-analysis/img/220px-Triangle.Centroid.svg.png?raw=1)

__Source:__ [Wikipedia](https://en.wikipedia.org/wiki/Centroid)

Below shows an example of creating a geodataframe of Nigeria's ADM2, where we use the centroid as the geometry.

In [ ]:
nga2_c_gdf = nga2_gdf.copy()
nga2_c_gdf['geometry'] = nga2_c_gdf['geometry'].centroid

nga2_c_gdf.plot()

#### Exercise 5: Create a roads object where (1) roads are buffered by 10 meters and (2) road types are dissolved, so the dataset contains one row per road type

#### 5a. Buffer the road network by 10 meters

#### 5b. Dissolve the road network by road type (ie, the "highway" variable)

### Spatial Operations: Using Multiple Datasets

#### Distances: Using Projected CRS

For this example, we'll compute the distance between each school to a motorway.

In [ ]:
#### Project so units are in meters
schools_proj_gdf = schools_gdf.to_crs(32632)
roads_proj_gdf   = roads_gdf.to_crs(32632)

In [ ]:
#### Road dataset of just motorways
roads_diss_gdf = roads_proj_gdf.dissolve('highway')
roads_diss_gdf = roads_diss_gdf.reset_index()

roads_motorway_gdf = roads_diss_gdf[roads_diss_gdf['highway'] == 'motorway']

In [ ]:
#### Distance
schools_proj_gdf['dist_to_road_m'] = schools_proj_gdf['geometry'].apply(lambda x: x.distance(roads_motorway_gdf['geometry'].iloc[0]))

schools_proj_gdf['dist_to_road_m'].describe()

#### Intersects (True/False vector)

For this example we'll determine which of Lagos's second administrative divisions intersects with a motorway.

First, let's check the CRS of the datasets we'll use and ensure that they're in the same CRS.

In [ ]:
roads_motorway_gdf.crs

In [ ]:
lagos_gdf.crs

In [ ]:
lagos_proj_gdf = lagos_gdf.to_crs(32632)

In [ ]:
## Create new variable whether ADM intersects with motorway
lagos_proj_gdf['inter_motorway'] = lagos_proj_gdf['geometry'].apply(lambda x: x.intersects(roads_motorway_gdf['geometry'].iloc[0]))

In [ ]:
lagos_proj_gdf.inter_motorway.head()

#### Overlay Data

We have roads for all of Lagos. Here we'll show how to use the `overlay` function to:

1. Create a new object of roads that are in a certain administrative region.
2. Create a new object of roads that are NOT in a certain administrative region.

In [ ]:
eo_proj_gdf = lagos_proj_gdf[lagos_proj_gdf.NAME_2 == "Eti-Osa"]

In [ ]:
## 1. Create a new object of roads that are in a certain administrative region.
roads_eo_gdf = roads_proj_gdf.overlay(eo_proj_gdf, how='intersection')

roads_eo_gdf.plot()

In [ ]:
## 2. Create a new object of roads that are NOT in a certain administrative region.
roads_not_eo_gdf = roads_proj_gdf.overlay(eo_proj_gdf, how='difference')

roads_not_eo_gdf.plot()

#### Spatial Join

We have a dataset of schools. The school GeoDataFrame contains information such as the school name, but not on the administrative region it's in. To add data on the administrative region that the school is in, we'll perform a spatial join

In [ ]:
schools_admdata_gdf = gpd.sjoin(
    schools_gdf,
    nga2_gdf,
    how='left',
    predicate="within",
)

schools_admdata_gdf.head()

#### Exercise 6: Determine the ADM2 name that the centroid of each road is in

#### 6a: Create a new roads object where each road geometry is the centroid of the road

#### 6b: Determine the ADM2 name that each road centroid is in

## Challenge Exercise

__Inaccessible schools: Create an interactive map of Lagos second administrative divisions that shows the number of schools that are greater than 1 kilometer of a trunk road or motorway__

In [ ]:
## Load data
nga2_gdf  = gpd.read_file(os.path.join("data", "gadm_nga_2.geojson"))
roads_gdf = gpd.read_file(os.path.join("data", "osm_lagos_roads.geojson"))

# Additional Resources

* https://aeturrell.github.io/coding-for-economists/geo-intro.html
* https://pygis.io/docs/d_access_osm.html